# Julia code (parts d, e)

In [1]:
#--- load packages
using JuMP
using Gurobi
using CSV
using DataFrames;

In [9]:
dcs = CSV.read("Dartboard_dcs.csv", DataFrame)
#future = CSV.read("Dartboard-future-predicts.csv", DataFrame)
historical = CSV.read("Dartboard_historical-1.csv", DataFrame) # file output from R program
;

In [10]:
historical[!,"UniqueCustomer"] = string.(historical[!,"County.Name"], ", ",  historical[!,"State.Name"])
#future[!,"UniqueCustomer"] = string.(future[!,"County.Name"], ", ",  future[!,"State.Name"])
;

In [11]:
select!(historical, Not("State.Name"))
select!(historical, Not("County.Name"))
select!(historical, Not("Year"))
select!(historical, Not("FIPS_Code"))

,Latitude,Longitude,Week,Income,Population,Week_Num,Season,Sales
,Float64,Float64,Int64,Float64,Float64,Int64,Int64,Float64
1,41.244,-73.363,1,1.06943e5,9.33226e5,1,1,8.58471e5
2,41.244,-73.363,2,1.06815e5,933351.0,2,1,8.16707e5
3,41.244,-73.363,3,1.06686e5,9.33476e5,3,1,706033.0
4,41.244,-73.363,4,1.06557e5,9.336e5,4,1,6.99396e5
5,41.244,-73.363,5,1.06429e5,9.33725e5,5,2,6.58245e5
6,41.244,-73.363,6,1.06301e5,9.33849e5,6,2,6.93406e5
7,41.244,-73.363,7,1.06173e5,933974.0,7,2,699856.0
8,41.244,-73.363,8,1.06045e5,9.34099e5,8,2,520851.0
9,41.244,-73.363,9,1.05917e5,9.34223e5,9,3,637070.0


In [17]:
#select(historical, Week_Num >= 147)
test = filter(row -> row.Week_Num > 146, historical)
train = filter(row -> row.Week_Num <= 146, historical)

,Latitude,Longitude,Week,Income,Population,Week_Num,Season,Sales
,Float64,Float64,Int64,Float64,Float64,Int64,Int64,Float64
1,41.244,-73.363,1,1.06943e5,9.33226e5,1,1,8.58471e5
2,41.244,-73.363,2,1.06815e5,933351.0,2,1,8.16707e5
3,41.244,-73.363,3,1.06686e5,9.33476e5,3,1,706033.0
4,41.244,-73.363,4,1.06557e5,9.336e5,4,1,6.99396e5
5,41.244,-73.363,5,1.06429e5,9.33725e5,5,2,6.58245e5
6,41.244,-73.363,6,1.06301e5,9.33849e5,6,2,6.93406e5
7,41.244,-73.363,7,1.06173e5,933974.0,7,2,699856.0
8,41.244,-73.363,8,1.06045e5,9.34099e5,8,2,520851.0
9,41.244,-73.363,9,1.05917e5,9.34223e5,9,3,637070.0


In [18]:
CSV.write("alldata_train.csv", train)
CSV.write("alldata_test.csv", test)

"alldata_test.csv"

In [5]:
counties = unique(future.UniqueCustomer)


765-element Vector{String}:
 "Fairfield County, Connecticut"
 "Hartford County, Connecticut"
 "Litchfield County, Connecticut"
 "Middlesex County, Connecticut"
 "New Haven County, Connecticut"
 "New London County, Connecticut"
 "Tolland County, Connecticut"
 "Windham County, Connecticut"
 "Kent County, Delaware"
 "New Castle County, Delaware"
 "Sussex County, Delaware"
 "District of Columbia, District of Columbia"
 "Adams County, Indiana"
 ⋮
 "Roane County, West Virginia"
 "Summers County, West Virginia"
 "Taylor County, West Virginia"
 "Tucker County, West Virginia"
 "Tyler County, West Virginia"
 "Upshur County, West Virginia"
 "Wayne County, West Virginia"
 "Webster County, West Virginia"
 "Wetzel County, West Virginia"
 "Wirt County, West Virginia"
 "Wood County, West Virginia"
 "Wyoming County, West Virginia"

In [5]:
future[!,"WeekNum"] = (future.Year .- 2015).*52 .+ future.Week
;

In [6]:
# q[j] > max DC j capacity (pallets)
q = floor.(dcs.Max_Size / 13.5)

20-element Vector{Float64}:
 88888.0
 88888.0
 66666.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0
 88888.0

In [7]:
# f[j] > fixed dc costs ($)
f = dcs.Fixed_Cost

20-element Vector{Int64}:
        0
        0
        0
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000
 25000000

In [9]:
# v[j] > variable dc costs ($ per pallet)
v = dcs.Variable_Cost * 13.5 # convert sqft to pallet

20-element Vector{Float64}:
    0.0
    0.0
    0.0
 1221.75
 1782.0
 1012.5
 1244.7
 2002.0500000000002
  927.45
 1387.8
 1009.8
 1023.3
 1595.7
 1242.0
 2247.75
 2119.5
 1062.45
 1188.0
  864.0
 1370.25

In [10]:
j_tot = length(q)
i_tot = length(counties)
t_tot = length(unique(future[!,"WeekNum"]))

130

In [11]:
# d[i,j] > distances from county i to DC j (miles)
using Distances
d = zeros(i_tot, j_tot)
for i=1:i_tot
    tt = filter(row -> row.LocationName in [counties[i]], future)[1,:]
    lat_county = tt.Latitude
    lon_county = tt.Longitude
    for j=1:j_tot
        lat_dc = dcs[j,:].Latitude
        lon_dc = dcs[j,:].Longitude

        r = haversine((lat_county,lon_county),(lat_dc,lon_dc), 3958.8)
        d[i, j] = r
    end
end
d

765×20 Matrix{Float64}:
 136.128   286.33    500.039   611.161   …  191.77    707.339   110.176
  91.067   332.308   544.71    656.087      234.69    751.883    64.2279
 124.509   299.397   511.259   622.635      201.471   718.441    97.4788
  78.3715  343.723   557.684   668.883      248.413   764.947    53.5053
 105.974   316.235   530.17    641.324      221.368   737.456    80.4705
  50.7757  371.145   585.46    696.652   …  276.034   792.723    28.9082
  62.7439  360.214   573.042   684.413      262.896   780.213    36.4287
  42.8454  379.777   592.938   704.296      282.754   800.112    18.1702
 284.516   137.438   357.036   466.528       79.5912  564.039   259.401
 293.211   128.967   346.332   456.241       67.5074  553.503   267.558
 279.824   142.77    364.102   473.15    …   89.5753  570.884   255.253
 391.609    34.1378  249.489   358.356       91.2882  456.09    365.621
 935.407   521.693   299.676   189.426      611.007    92.6104  907.866
   ⋮                             

In [12]:
# n[i,t] > demand of county i for week t (pallets)
n = zeros(i_tot, t_tot) # each stores weekly capacity needs
for i=1:i_tot
    tt = filter(row -> row.LocationName in [counties[i]], future).Sales
    n[i,:] = tt / 1000 # convert $ to pallets
end
n

765×130 Matrix{Float64}:
 1595.88    1603.42    1604.14    …  3153.92    3168.6     3183.34
 1726.2     1734.28    1734.96       3408.74    3424.67    3440.68
  355.624    357.25     357.353       688.283    691.368    694.466
  310.716    312.172    312.297       612.502    615.349    618.209
 1711.8     1719.83    1720.53       3366.77    3382.31    3397.92
  532.528    534.992    535.176   …  1042.54    1047.33    1052.15
  300.909    302.297    302.396       593.074    595.858    598.655
  242.092    243.205    243.283       475.447    477.66     479.884
  370.973    372.754    372.947       747.137    750.764    754.408
 1108.94    1114.26    1114.84       2202.47    2212.75    2223.08
  436.487    438.779    439.203   …   924.921    929.753    934.609
 1222.82    1228.94    1229.83       2499.17    2511.44    2523.77
   76.4414    76.8053    76.8419      152.812    153.543    154.277
    ⋮                             ⋱                        
   32.2544    32.3966    32.4006     

In [13]:
# tr[i,t] > demand of county i for week t (trucks)
tr = ceil.(n / 20) 

765×130 Matrix{Float64}:
 80.0  81.0  81.0  81.0  81.0  82.0  …  157.0  157.0  158.0  159.0  160.0
 87.0  87.0  87.0  88.0  88.0  88.0     170.0  170.0  171.0  172.0  173.0
 18.0  18.0  18.0  18.0  19.0  19.0      35.0   35.0   35.0   35.0   35.0
 16.0  16.0  16.0  16.0  16.0  16.0      31.0   31.0   31.0   31.0   31.0
 86.0  86.0  87.0  87.0  87.0  88.0     168.0  168.0  169.0  170.0  170.0
 27.0  27.0  27.0  27.0  28.0  28.0  …   52.0   52.0   53.0   53.0   53.0
 16.0  16.0  16.0  16.0  16.0  16.0      30.0   30.0   30.0   30.0   30.0
 13.0  13.0  13.0  13.0  13.0  13.0      24.0   24.0   24.0   24.0   24.0
 19.0  19.0  19.0  19.0  19.0  19.0      38.0   38.0   38.0   38.0   38.0
 56.0  56.0  56.0  57.0  57.0  57.0     110.0  110.0  111.0  111.0  112.0
 22.0  22.0  22.0  23.0  23.0  23.0  …   46.0   47.0   47.0   47.0   47.0
 62.0  62.0  62.0  62.0  63.0  63.0     124.0  125.0  125.0  126.0  127.0
  4.0   4.0   4.0   4.0   4.0   4.0       8.0    8.0    8.0    8.0    8.0
  ⋮          

In [14]:
# w[i] > max 8-week demand of county i (pallets)
week8_cap = zeros(i_tot)
for i=1:i_tot
    maxi=0
    for t=1:t_tot
        week8_tot = sum(n[i,t:min(t_tot,(t+7))]) # what each county i needs for next 8 weeks
        maxi = max(week8_tot, maxi)
    end
    week8_cap[i] = maxi
end
w = week8_cap

765-element Vector{Float64}:
 25073.75951672689
 27098.67496628126
  5473.234040172787
  4869.432446850307
 26767.384557885092
  8288.93262657163
  4714.653161489682
  3779.795321982008
  5937.999783555015
 17509.2146529355
  7346.979544313601
 19860.97409201947
  1214.6249817856103
     ⋮
   494.00315
   458.97065000000003
   574.63995
   229.13484
   301.56412
   867.2682
  1419.9581
   305.1487
   535.8682
   208.20829
  2854.9215000000004
   758.05057

In [15]:
model = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attribute(model, "OutputFlag", 0)

Academic license - for non-commercial use only - expires 2022-08-19


0

In [16]:
@variable(model, k[1:j_tot] >= 0, Int) # dc capacity
@variable(model, z[1:j_tot], Bin) # which dc to select
@variable(model, x[1:i_tot, 1:j_tot], Bin) # which county i is serviced by DC j
;

In [17]:
@constraint(model, [j in 1:j_tot], k[j] <= q[j]*z[j]) # each DC j must have no more than max DC j capacity
#@constraint(model, [j in 1:j_tot], 5*sum(k[j] for j=1:j_tot) >= Q) # total DC j capacity >= maximum forecasted 8-week capacity
@constraint(model, [j in 1:j_tot], sum(w[i] * x[i,j] for i=1:i_tot) <= 5*k[j]) # each DC j must have enough 8-week capacity for the i counties it services
@constraint(model, [i in 1:i_tot], sum(x[i,j] for j in 1:j_tot) == 1) # each county i must be serviced by 1 single DC j

@constraint(model, z[1] == 1)
@constraint(model, z[2] == 1)
@constraint(model, z[3] == 1)

@constraint(model, k[1] == q[1])
@constraint(model, k[2] == q[2])
@constraint(model, k[3] == q[3])
;

In [18]:
@objective(model, Min, sum(z[j]*f[j] + k[j]*v[j] for j = 1:j_tot) + 1.55*sum((x[i,j]*d[i,j]*tr[i,t]) for i = 1:i_tot, j = 1:j_tot, t = 1:t_tot))
optimize!(model)
;

In [19]:
cost = objective_value(model)
dc_used = value.(z)
county_dc_mapping = value.(x)
dc_capacities = value.(k)
;

In [20]:
cost

7.585953200767107e8

In [21]:
dc_used

20-element Vector{Float64}:
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0

In [22]:
county_dc_mapping

765×20 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [23]:
dc_capacities

20-element Vector{Float64}:
 88888.0
 88888.0
 66666.0
     0.0
     0.0
     0.0
     0.0
     0.0
     0.0
     0.0
 88884.0
     0.0
     0.0
 51901.0
     0.0
     0.0
 88885.0
 88888.0
 88886.0
     0.0

# d)

In [24]:
# i)

p=1
for num in dc_used
    if num == 1
        println("New DC: ", dcs.Location[p])
    end
    p=p+1
end

New DC: Providence
New DC: Richmond
New DC: Youngstown
New DC: Kalamazoo
New DC: Lancaster
New DC: Scranton
New DC: Syracuse
New DC: Toledo


In [25]:
# ii)

p=1
for num in dc_used
    if num == 1
        println("DC: ", dcs.Location[p])
        println("Counties served: ", sum(county_dc_mapping[:,p]))
    end
    p=p+1
end

DC: Providence
Counties served: 57.0
DC: Richmond
Counties served: 118.0
DC: Youngstown
Counties served: 121.0
DC: Kalamazoo
Counties served: 222.0
DC: Lancaster
Counties served: 42.0
DC: Scranton
Counties served: 27.0
DC: Syracuse
Counties served: 59.0
DC: Toledo
Counties served: 119.0


In [26]:
# iii)

fixed_cost = 0
var_cost = 0
p=1
for num in dc_used
    if num == 1
        #println("DC: ", dcs.Location[p], " | Fixed Cost: ", dcs.Fixed_Cost[p], " | Var Cost: ", dcs.Variable_Cost[p]*dc_capacities[p])
        fixed_cost += f[p]
        var_cost += v[p]*dc_capacities[p]
    end
    p=p+1
end

constr_cost = fixed_cost+var_cost
println("Total Construction Cost = ", constr_cost)

Total Construction Cost = 5.5604842145e8


In [27]:
trans_cost = 1.55*sum((county_dc_mapping[i,j]*d[i,j]*tr[i,t]) for i = 1:i_tot, j = 1:j_tot, t = 1:t_tot)
println("Total Transportation Cost = ", trans_cost)

Total Transportation Cost = 2.025468986267158e8


In [28]:
trans_cost + constr_cost

7.585953200767158e8

# e)

In [29]:
# Optimize for construction cost first

model = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attribute(model, "OutputFlag", 0)

@variable(model, z[j = 1:j_tot], Bin)
@variable(model, k[i = 1:j_tot] >=0, Int)

@constraint(model, [j = 1:j_tot], k[j] <= q[j]*z[j])
@constraint(model, [i = 1:i_tot, j = 1:j_tot], 5*sum(k[j] for j = 1:j_tot) >= sum(w[i] for i = 1:i_tot))

@objective(model, Min, sum(z[j] * (f[j] + (v[j]*k[j])) for j = 1:j_tot))

optimize!(model)

cost_e = objective_value(model)
dc_cheapest = value.(z)
dc_cheapest_cap = value.(k)
;

Academic license - for non-commercial use only - expires 2022-08-19


In [30]:
cost_e

5.170813208e8

In [31]:
dc_cheapest

20-element Vector{Float64}:
  1.0
  1.0
  1.0
 -0.0
 -0.0
  1.0
 -0.0
 -0.0
  1.0
 -0.0
  1.0
  1.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  1.0
 -0.0

In [33]:
dc_cheapest_cap

20-element Vector{Float64}:
 88888.0
 88888.0
 66666.0
     0.0
    -0.0
 88888.0
     0.0
    -0.0
 88888.0
    -0.0
 88888.0
 51876.0
    -0.0
     0.0
    -0.0
    -0.0
     0.0
     0.0
 88888.0
    -0.0

In [34]:
# i)

fixed_cost = 0
var_cost = 0
p=1
for num in dc_cheapest
    if num == 1
        println("DC: ", dcs.Location[p], " | Fixed Cost: ", dcs.Fixed_Cost[p], " | Var Cost: ", dcs.Variable_Cost[p]*dc_cheapest_cap[p])
        fixed_cost += f[p]
        var_cost += v[p]*dc_cheapest_cap[p]
    end
    p=p+1
end

constr_cost = fixed_cost+var_cost
println(" ")
println("Total Construction Cost = ", constr_cost)

DC: Providence | Fixed Cost: 0 | Var Cost: 0.0
DC: Richmond | Fixed Cost: 0 | Var Cost: 0.0
DC: Youngstown | Fixed Cost: 0 | Var Cost: 0.0
DC: Bangor | Fixed Cost: 25000000 | Var Cost: 6.6666e6
DC: Chillicothe | Fixed Cost: 25000000 | Var Cost: 6.106605600000001e6
DC: Kalamazoo | Fixed Cost: 25000000 | Var Cost: 6.648822399999999e6
DC: Knoxville | Fixed Cost: 25000000 | Var Cost: 3.9322008e6
DC: Toledo | Fixed Cost: 25000000 | Var Cost: 5.688832e6
 
Total Construction Cost = 5.170813208e8


In [35]:
dc_cheapest_bin = dc_cheapest .== 1

20-element BitVector:
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 1
 0
 0
 0
 0
 0
 0
 1
 0

In [ ]:
model = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attribute(model, "OutputFlag", 0)

@variable(model, x[1:i_tot, 1:j_tot], Bin) # which county i is serviced by DC j


@constraint(model, [j in 1:j_tot], sum(w[i] * x[i,j] for i=1:i_tot) <= 5*dc_cheapest_cap[j]) # each DC j must have enough 8-week capacity for the i counties it services
@constraint(model, [i in 1:i_tot], sum(x[i,j] for j in 1:j_tot) == 1) # each county i must be serviced by 1 single DC j

@objective(model, Min, 1.55*sum((x[i,j]*d[i,j]*tr[i,t]) for i = 1:i_tot, j = 1:j_tot, t = 1:t_tot))
optimize!(model)

county_dc_mapping = value.(x)
trans_cost2 = objective_value(model)